In [9]:
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
import numpy as np


In [10]:
# fetch data

x_train = pd.read_csv("AMF_train_X.csv")
y_train = pd.read_csv("AMF_train_Y.csv")
x_test = pd.read_csv("AMF_test_X.csv")


FileNotFoundError: [Errno 2] File AMF_train_X.csv does not exist: 'AMF_train_X.csv'

In [ ]:
# dictionnary {Trader:TYPE}

type_d = {}
for index, row in y_train.iterrows():
    type_d[row['Trader']] = row['type']

In [ ]:
# {Custom train data}

#x_train2 = x_train.iloc[:60467,:]
#x_train2 = x_train2.dropna()
#y_train2 = [type_d[x] for x in x_train2['Trader']]
#x_train2 = x_train2.drop(columns=['Index','Share','Day','Trader'])

#x_test2 = x_train.iloc[60467:,:]
#x_test2 = x_test2.dropna()

#y_test2 = [type_d[x] for x in x_test2['Trader']]
#x_test2 = x_test2.drop(columns=['Index','Share','Day','Trader'])

x_train2 = x_train.fillna(method='ffill')
x_train2 = x_train2.fillna(method='bfill')
y_train2 = [type_d[x] for x in x_train2['Trader']]
x_train2 = x_train2.drop(columns=['Index','Share','Day','Trader'])


In [ ]:
#svclassifier = SVC(kernel='linear')
#svclassifier.fit(x_train2, y_train2)

sgd = SGDClassifier()
sgd.fit(x_train2, y_train3)

In [ ]:
# test

#y_pred = svclassifier.predict(x_test2)
#print(classification_report(y_test2,y_pred))


x_test2 = x_test.dropna()
x_test2 = x_test2.drop(columns=['Index','Share','Day','Trader'])
y_pred = sgd.predict(x_test2)

In [ ]:
x_test2 = x_test.fillna(method='ffill')
x_test2 = x_test2.fillna(method='ffill')

names = x_test2['Trader'].unique()

convention = {'HFT': 0, 'NON HFT': 1, 'MIX': 2} 
result = {e: [0,0,0] for e in names}
i = 0
for index, row in x_test2.iterrows():
    result[row['Trader']][convention[y_pred[i]]] += 1
    i+=1

data=[]
for trader in names:
    n = sum(result[trader])
    if result[trader][0]>0.85*n:
        data.append([trader, 'HFT'])
    elif result[trader][2]>0.5*n:
        data.append([trader, 'MIX'])
    else:
        data.append([trader, 'NON HFT'])

export_df = pd.DataFrame(data, columns = ['Trader', 'type'])
export_df.to_csv(r'test.csv',index=False)


In [ ]:
df = pd.read_csv("AMF_train_Y.csv")
for name in [e for e in df['Trader'].unique() if not(e in names)]:
    df = df.drop( df[df.Trader==name].index)
export_df.to_csv(r'test_y.csv',index=False)